In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time

In [123]:
url = 'https://www.usatoday.com/sports/mlb/salaries/'

In [124]:
html = requests.get(url)

In [125]:
html

<Response [200]>

In [126]:
soup = BeautifulSoup(html.content, 'html.parser')

In [168]:
rankings_container = soup.find('tbody')

In [170]:
names = rankings_container.findAll('td', class_ = 'player_display')
first = names[0].get_text()

In [171]:
first

'\n                                            Max Scherzer\n                                        '

In [216]:
first = first.replace('\n', "")
first = first.replace(" ", "")

In [217]:
first

'MaxScherzer'

In [215]:
teams = rankings_container.findAll('td', class_ = 'team_abbr')
first_team = teams[0].find('a').get_text()
first_team = first_team.replace('\n', "")
first_team = first_team.replace(" ", "")
first_team

'WSH'

In [214]:
positions = rankings_container.findAll('td', class_ = 'position')
first_position = positions[0].get_text()
first_position = first_position.replace('\n', "")
first_position = first_position.replace(" ", "")
first_position

'SP'

In [219]:
salaries = rankings_container.findAll('td', class_ = 'salary')
first_salary = salaries[0].get_text()
first_salary = first_salary.replace('\n', "")
first_salary = first_salary.replace(' ', "")
first_salary

'$42,142,857'

In [248]:
names = []

def player_names(rankings_container):
#     names = []
    for item in rankings_container:
#         print(item)
        name = item.find('td', class_='player_display')
#         print(name)
        if name:
            name = name.get_text()
            name = name.replace('\n', "")
            name = name.replace(" ", "")
            names.append(name)
        else:
            names.append("NaN")
    return names

names = player_names(rankings_container)

In [249]:
teams = []

def all_teams(rankings_container):
    for item in rankings_container:
        team = item.find('td', class_='team_abbr')
        if team:
            team = team.get_text()
            team = team.replace('\n', "")
            team = team.replace(' ', "")
            teams.append(team)
        else:
            teams.append("NaN")
    return teams

teams = all_teams(rankings_container)

In [250]:
positions = []

def all_positions(rankings_container):
    for item in rankings_container:
        position = item.find('td', class_='position')
        if position:
            position = position.get_text()
            position = position.replace('\n', "")
            position = position.replace(' ', "")
            positions.append(position)
        else:
            positions.append("NaN")
    return positions

postions = all_positions(rankings_container)

In [251]:
salaries = []

def all_salaries(rankings_container):
    for item in rankings_container:
        salary = item.find('td', class_='salary')
        if salary:
            salary = salary.get_text()
            salary = salary.replace('$', "")
            salary = salary.replace(',', "")
            salary = salary.replace('\n', "")
            salary = salary.replace(' ', "")
            salaries.append(int(salary))
        else:
            salaries.append("NaN")
    return salaries


salaries = all_salaries(rankings_container)

In [252]:
player_info = {'name': names, 'team': teams, 'position': positions, 'salary': salaries}

In [253]:
df = pd.DataFrame(player_info)

In [254]:
df.head()

,name,team,position,salary
0,MaxScherzer,WSH,SP,42142857
1,StephenStrasburg,WSH,SP,36428571
2,MikeTrout,LAA,CF,34083333
3,ZackGreinke,ARI,SP,32421884
4,DavidPrice,BOS,SP,31000000


In [255]:
df.shape

(877, 4)